In [ ]:
#| default_exp experimental

# experimental

> Experimental `dialoghelper` capabilities.

In [ ]:
#| export

import pkg_resources
import uuid
from fasthtml.common import Div, Script
from time import sleep
import base64
import json
import time
from dialoghelper.core import *
from dataclasses import dataclass
from httpx import get as xget, post as xpost

In [ ]:
#| export

_js_loaded = False

def _load_screenshot_js():
    "Load screenshot capability if not already present."
    global _js_loaded
    if _js_loaded:
        print("screenshot.js already loaded")
        return
    js_content = pkg_resources.resource_string('dialoghelper', 'screenshot.js').decode('utf-8')
    add_html(Div(Script(js_content), hx_swap_oob='beforeend:#js-script'))
    print("Added screenshot.js")
    _js_loaded = True

In [ ]:
#| export

def create_iife(code: str) -> str:
    "Wrap javascript code string in an IIFE"
    return f'''
(async () => {{
{code}
}})();
'''

In [ ]:
#| export

@dataclass
class ToolImageResult:
    data: bytes
    media_type: str = "image/jpeg"
    alt_text: str = "Tool generated image"

In [ ]:
#| export

def capture_screen(add_note=False):
    "Capture current desktop/screen and insert as image in dialog, returning the screenshot as a ToolImageResult which will be sent back to the API is an image. `persist` determines if the screenshot is added as a note"
    _load_screenshot_js()        
    sid = add_msg("[Viewing Screen]", mt="note")
    screenshot_code = f'captureScreenAndUpload("{sid}");'    
    trigger_script = create_iife(screenshot_code)
    add_html(Div(Script(trigger_script), hx_swap_oob=f'beforeend:#js-script'))
    max_attempts = 50
    for attempt in range(max_attempts):
        result = xpost('http://localhost:5001/pop_data_', data={'data_id': sid})
        if result.status_code == 200:
            try:
                if not result.text.strip():
                    time.sleep(0.5)
                    continue
                data = result.json()
                assert data is not None
                parsed_data = json.loads(data) if isinstance(data, str) else data
                print(f"Parsed data: {parsed_data}")
                if 'error' in parsed_data: return f"Screenshot failed: {parsed_data['error']}"
                if 'data' in parsed_data:
                    inner_data = json.loads(parsed_data['data'])
                    if 'img_data' in inner_data and 'img_type' in inner_data:
                        if not add_note: del_msg(sid)
                        return ToolImageResult(data=inner_data['img_data'], # b64 data
                                               media_type=inner_data['img_type'],
                                               alt_text=f"Screenshot captured ({inner_data.get('size', 'unknown')} bytes)")
                return f"Screenshot data incomplete: {parsed_data}"
            except json.JSONDecodeError as e: print(f"JSON decode error: {e}, Raw text: '{result.text}'")
            except Exception as e: print(f"Error processing screenshot data: {e}")
        time.sleep(0.5)
    return "Screenshot capture timed out"